In [1]:
import psycopg2
import csv
from datetime import date
conn = psycopg2.connect(dbname="nate", user="nate")
cur = conn.cursor()


In [2]:
with open('ign.csv') as f:
    next(f)
    reader = csv.reader(f)
    data = [row for row in reader]
        

In [3]:
len(data)

18625

In [4]:
data[0]

['5249979066121302517',
 'Amazing',
 'LittleBigPlanet PS Vita',
 '/games/littlebigplanet-vita/vita-98907',
 'PlayStation Vita',
 '9.0',
 'Platformer',
 'Y',
 '2012',
 '9',
 '12']

In [5]:
updated_data = []
for row in data:
    updated_row = row[:8]
    updated_row.append(date(int(row[8]), int(row[9]), int(row[10])))
    updated_data.append(updated_row)
data = updated_data


In [6]:
labels = {
    'id': 0,
    'score_phrase': 1,
    'title': 2,
    'url': 3,
    'platform' : 4,
    'score': 5,
    'genre': 6,
    "editors_choice": 7
    
}

In [7]:
labels

{'id': 0,
 'score_phrase': 1,
 'title': 2,
 'url': 3,
 'platform': 4,
 'score': 5,
 'genre': 6,
 'editors_choice': 7}

In [8]:
def column_max(col):
    max_score = 0
    for row in data:
        if len(row[col]) > max_score:
            max_score = len(row[col])
    return max_score
    

In [9]:
scores = {}
for key, value in labels.items():
    scores[key] = column_max(value)

In [10]:
scores

{'id': 19,
 'score_phrase': 11,
 'title': 96,
 'url': 105,
 'platform': 20,
 'score': 4,
 'genre': 25,
 'editors_choice': 1}

In [11]:
cur.execute('DROP TABLE IF EXISTS ign_reviews')

In [12]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS ign_reviews (
        id BIGINT PRIMARY KEY,
        score_phrase VARCHAR(11),
        title VARCHAR(96),
        url VARCHAR(105),
        platform VARCHAR(20),
        score DECIMAL(3,1),
        genre VARCHAR(25),
        editors_choice BOOLEAN,
        date DATE
        
    )
""")
conn.commit()

In [13]:
for row in data:
    cur.execute('INSERT INTO ign_reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s,  %s)', row)
    
conn.commit() 

In [14]:
cur.execute('SELECT * FROM ign_reviews')

In [15]:
table = cur.fetchall()
print(table[:3])

[(5249979066121302517, 'Amazing', 'LittleBigPlanet PS Vita', '/games/littlebigplanet-vita/vita-98907', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12)), (7399589116837456607, 'Amazing', 'LittleBigPlanet PS Vita -- Marvel Super Hero Edition', '/games/littlebigplanet-ps-vita-marvel-super-hero-edition/vita-20027059', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12)), (582057716445789124, 'Great', 'Splice: Tree of Life', '/games/splice/ipad-141070', 'iPad', Decimal('8.5'), 'Puzzle', False, datetime.date(2012, 9, 12))]


In [16]:
#conn.close()